In [2]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

C:\ProgramData\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
df_short = pd.read_csv("data/processed/processed_short.csv")
df_medium = pd.read_csv("data/processed/processed_medium.csv")
df_dank = pd.read_csv("data/processed/processed_dank.csv")

In [4]:
def generate_vectors(df):
    name =[x for x in globals() if globals()[x] is df][0]
    df['list'] = df["text"].str.split()
    documents = df["list"].to_numpy()
    skipgram = Word2Vec(sentences=documents, vector_size=100, window=5, sg=1)
    word_vectors = skipgram.wv
    word_vectors.save("data/" + str(name) + ".wordvectors")

# Generating and saving vectors

In [15]:
generate_vectors(df_short)

In [22]:
generate_vectors(df_medium)

In [5]:
generate_vectors(df_dank)

# How to load them:

In [6]:
wv = KeyedVectors.load("data/df_dank.wordvectors", mmap='r')

In [7]:
wv['meme']

array([ 4.4713367e-02,  2.4307360e-01, -1.5273180e-02, -3.5014700e-02,
       -5.3334620e-02, -3.9143988e-01,  3.1298506e-01,  3.4044915e-01,
       -2.1864204e-01, -4.0944170e-02,  4.4334292e-02, -1.7739116e-01,
       -3.3340387e-02,  2.2267476e-02,  1.3503331e-01, -6.7491449e-02,
        3.3322969e-01, -4.3420917e-01, -3.7479603e-01, -7.4566668e-01,
        2.2543620e-01,  1.8572259e-01,  6.2628455e-02,  1.7958254e-02,
       -5.5491358e-02,  4.0784862e-02, -2.2691788e-01, -4.7135953e-02,
       -1.9822851e-01,  9.9093206e-02,  2.2066329e-01,  2.7250946e-01,
        8.6985119e-03, -1.4721279e-01, -2.5610405e-01,  5.0144482e-01,
        2.0944414e-01, -5.9222382e-02, -2.6948076e-01, -1.8638583e-01,
       -4.9734455e-02,  4.3178431e-04, -3.3925262e-01, -7.5807802e-02,
        1.3481711e-02, -6.0152500e-03, -3.6021152e-01,  4.2551942e-02,
        2.3714823e-01,  3.5974035e-01, -2.1724962e-01, -1.4240047e-01,
       -8.3204523e-02,  8.4656268e-02, -1.3767689e-01,  9.5627904e-02,
      